In [1]:
import os
import dpkt
import cv2
import numpy as np
import pandas as pd

from sklearn.cluster import DBSCAN
from DataReader import LidarLoader

from DetectedObject import DetectedObject
from LidarDBSCAN import AdaptiveDBSCAN


In [3]:

class FrameGen():
    def __init__(self,frame_path,detecting_range,bck_voxel_path,with_bf = True):
        self.frame_path = frame_path
        self.bck_voxel_path = bck_voxel_path
        self.with_bf = with_bf
        self.detecting_range = detecting_range
        
    def extract_frame_dic(self,point_cloud,Adb):
        if len(point_cloud) == 0:
            return {}
        label = Adb.fit_predict(point_cloud)# cluster part point_cloud: 2D array [[2.1,2.3],[2.2,3.4],[x,y],[100,121]] label:[0,0,1,-1]
        uniq_label = np.unique(label)
        if -1 in uniq_label:
            uniq_label = uniq_label[uniq_label!=-1]
        frame_dic = {}
        for ind in range(len(uniq_label)):
            include_point = point_cloud[label == uniq_label[ind]].astype(np.float32)[:,[0,1]]
            rect = cv2.minAreaRect(include_point)
            center = rect[0]
            box_corner = cv2.boxPoints(rect)
            elevation_intensity = point_cloud[label == uniq_label[ind]].astype(np.float32)[:,[3,4]]
            detected_obj = DetectedObject(np.array(center),include_point,box_corner,elevation_intensity)
            frame_dic[ind] = detected_obj

        return frame_dic #next_frame

    def DBSCAN_pcap_frame_generator(self,eps,min_samples):
        db = DBSCAN(eps=eps,min_samples=min_samples)
        lidar_reader = LidarLoader(self.frame_path,self.bck_voxel_path,self.detecting_range,self.with_bf)
        frame_gen = lidar_reader.frame_gen()
        while True:
            while True:
                point_cloud = next(frame_gen)[:,[1,2,4,3,5]] # X,Y,D,Z,I
                frame_dic = self.extract_frame_dic(point_cloud,db)
                yield frame_dic

    def ADBSCAN_pcap_frame_generator(self,beta,min_sample_1,min_sample_2,min_sample_3):
        Adb = AdaptiveDBSCAN(beta,min_sample_1,min_sample_2,min_sample_3)
        lidar_reader = LidarLoader(self.frame_path,self.bck_voxel_path,self.detecting_range,self.with_bf)
        frame_gen = lidar_reader.frame_gen()
        while True:
            while True:
                point_cloud = next(frame_gen)[:,[1,2,4,3,5]] # X,Y,D,Z,I
                frame_dic = self.extract_frame_dic(point_cloud,Adb)
                yield frame_dic

In [6]:
pcap_file_path = 'C:/Users/super/OneDrive/Documents/2021/Lidar/bf1VER2.pcap'
# reset frame gen
detecting_range = 40 #meter
bck_voxel_path = 0
eps = 1
min_samples = 7

frame_gen = FrameGen(pcap_file_path,detecting_range,bck_voxel_path,with_bf=False).DBSCAN_pcap_frame_generator(eps,min_samples)
frame_ind = 1 # keep track of the frame number 

In [7]:
# step 1
ind_temp = 0
xmin=-200
ymin=-200
xmax= 200
ymax=200
forwardmap={}
backmap={}
for ix in range(int(xmin), int(xmax)):
    for iy in range(int(ymin), int(ymax)):
        myvec = []
        myvec.append(ix)
        myvec.append(iy)
        forwardmap[ind_temp] = myvec
        backmap[(ix, iy)]= ind_temp
        ind_temp=ind_temp+1

In [ ]:
# run algorithm on input

# test initializing

import csv
import os
from collections import defaultdict
import matplotlib.pyplot as plt
import numpy as np


# december highest 
# step 4 b 

# output the cost for each cluster pair 

class frequency_grid_cost(object):
    
    def __init__(self): # initialize forward and back map outside of init
        #self.numpoints = numpoints
        self.time_elapsed= 0 # same as index
        self.numcells = 160000
        
        self.thres = 10 # for hybrid 2 method
        self.angles = [] 
        
        self.xmin =-200
        self.xmax = 200
        self.ymin =-200
        self.ymax =200
        
        self.continueboolean = True # boolean
        
        self.forwardmap = {}  # dcmap1
        self.backmap = {}
        
        ind_temp = 0
        for ix in range(int(self.xmin), int(self.xmax)):
            for iy in range(int(self.ymin), int(self.ymax)):
                myvec = []
                myvec.append(ix)
                myvec.append(iy)
                self.forwardmap[ind_temp] = myvec
                self.backmap[(ix, iy)]= ind_temp
                ind_temp=ind_temp+1
                
        self.countmap = {}
        
        self.current_xposition=[]
        self.current_yposition=[]
        self.current_position = []
        self.prevmap = {} # set to current map at end of frame 
        self.currentmap = {}
        
        self.tracking_list = {} # list of objects that are tracked 
        
        self.range = 10
        
        # initialize grid count map
        self.countmap = {}
        
        # tracked array?
        self.trackinglist = {}
        # self positions in (x,y)
        self.xposition = []
        self.yposition = []
        self.position = [] 
        
        # data structure to hold multiple assignments 
        self.multimap_x = {} # maps the trajectory i to x values
        self.multimap_y = {} 
        self.multi_trackinglist = {} # maps the trajectory i to array of cluster_ids
        # map back most recent cluster index to 1st cluster index 
        self.most_recent_map = {}
    
        
    # set up based on trajectories file : training method 
    def setup_grid(self): 
        
        fileind = 1
        for filename in os.listdir('24hrdata'):
            print("file", filename)
            fname = '24hrdata/'+filename
            fileind =fileind+1
            #irow=0
            obnum=1
            with open(fname) as csv_file:
                csv_reader = csv.reader(csv_file, delimiter=',')
                line_count = 0
                for row in csv_reader:
                    trajectory_num = row[0]
                    if line_count==0:
                        line_count = line_count+1 # skip header
                        continue
                    if line_count==1:
                        prevrow = row
                        prevx = float(prevrow[6])
                        prevy = float(prevrow[7])
                        pfx = round(prevx)
                        pfy = round(prevy)
                        line_count = line_count+1
                        continue
                    currentx = float(row[6])
                    currenty = float(row[7])
                    fx = round(currentx)
                    fy = round(currenty)
                    if pfx == fx and pfy ==fy:
                        #prevframe = frameindex
                        continue
                    if obnum != trajectory_num:
                        pfx = fx
                        pfy = fy
                        obnum = trajectory_num
                        continue
                    #save
                    fromi = self.backmap[(pfx, pfy)]
                    toi = self.backmap[(fx, fy)]
                    #print("fromi is", fromi)
                    #print("toi is", toi)
                    if abs(pfx - fx)>10 or abs(pfy - fy)>10:
                        continue
                    # check if it is none 
                    val = self.countmap.get((fromi, toi))
                    if val == None:
                        #print("none val for fromi", fromi,"to toi ", toi)
                        self.countmap[(fromi, toi)] = 1
                    else:
                        
                        mcount = self.countmap[(fromi, toi)]
                        self.countmap[(fromi, toi)] = mcount+1
                        # check by printing
                        if mcount >600:
                            print("countmap from i: ", fromi, " toi:", toi)
                    pfx=fx
                    pfy=fy
    
    def highestfreq(self, fromi):
        highest = 0
        indexhighest = fromi
        (px, py) = self.forwardmap[fromi]
        for j in range(-10, 11):
            jx = px+j
            if jx>self.xmax-1 or jx<self.xmin: # check if pts in range
                continue
            for k in range(-10, 11):
                jy = py+k
                # check if pts are in range
                if jy>self.ymax-1 or jy<self.ymin:
                    continue
                toi = self.backmap[(jx, jy)]
                # check if value is none 
                val = self.countmap.get((fromi, toi))
                if val == None:
                    t = 0
                else:
                    t = self.countmap[(fromi, toi)]
                #if t>0:
                    #print(t)
                if t > highest:
                    highest=t
                    indexhighest=toi
        return highest, indexhighest

    def set_first_coordinates(self, frame_gen, init_cluster_num): # change init_cluster to init_cluster_num
        
        
        # update the first position
        first_frame = frame_gen #next(frame_gen) change it to not have next, to be consistent
        lenframe = len(first_frame)
        
        # loop to initialize x position, y position, cluster id 
        for i in range(0, init_cluster_num):
            ptcloud = first_frame[i].point_cloud
            xvals = []
            yvals = []
            currentmap={}
            for p in ptcloud:
                px =p[0]
                py =p[1]
                xvals.append(px)
                yvals.append(py)
            self.multimap_x[i] = [xvals]
            self.multimap_y[i] = [yvals]
            self.multi_trackinglist[i] = i 
            
            
        #self.current_xposition = xvals
        #self.current_yposition = yvals
        
        #self.trackinglist[0] = init_cluster
        self.continueboolean = True
    
    def predict(self, frame_gen, init_cluster_num): # ind: current index
        
        
        # frame gen is 
        next_frame = frame_gen #next(frame_gen)
        #print("len of next frame", len(next_frame))
        lenframe = len(next_frame)
        
        '''
        mf = defaultdict(list)
        
        mx = lenframe
        for j in range(0, mx):
            mf[j] = 0
            
        matchfreq = mf
        
        f= 0
        currentmap = {} # temporary currentmap
        currentmap_freq = {} # holds freq scores
        totalmap = {}
        '''
        
        xvalues=[]
        yvalues=[]
        #hxvalues = []
        #hyvalues = []
        
        # map of x, y values in next frame
        next_x = {}
        next_y = {}
        
        for j in range(0, init_cluster_num):
            pxmap = self.multimap_x[j]
            pymap = self.multimap_y[j]
            px = np.mean(pxmap)
            py = np.mean(pymap) 
            # take random sample of points, additionally
            array_j = []
            # calculate freq score to each cluster in next frame
            for i in range(0, lenframe):
                current_score = 0
                ptcloud = next_frame[i].point_cloud
                xvalues=[]
                yvalues=[]
                for p in ptcloud:
                    # extract x and y values
                    px =p[0]
                    py =p[1]
                    xvalues.append(px)
                    yvalues.append(py)
                    #print("pc of x", px)
                    #print("pc of y", py)
                    xr = round(px)
                    yr = round(py)
                    # calculate the freq from previous xval to xr , previous yval to yr 
                    toi = self.backmap[(px, py)]
                    fromi = self.backmap[(xr, yr)]
                    
                    val = self.countmap.get((fromi, toi))
                    if val == None:
                        t = 0
                    else:
                        t = self.countmap[(fromi, toi)]
                        
                    #h1, i1 = self.highestfreq(fromi)
                    # use frequency h1 
                    current_score = current_score + t
                # add to next_x, next_y
                if j==0:
                    next_x[i] = xvalues
                    next_y[i] = yvalues
                # add to array
                array_j[i] = current_score 
            # append to cost matrix
            if j==0:
                cm = array_j
            else:
                cm = np.vstack([cm, array_j])
        
        # linear sum assignment step 

        # created matrix with each row vector
        
        row_ind, col_ind = linear_sum_assignment(cm)
        
        # translate to the multimaps
        smost_recent_map = {}
        for j in range(0, init_cluster_num):
            r = row_ind[j]
            c = col_ind[j]
            self.multi_trackinglist[r] = c
            sx = self.multimap_x[r] 
            sx.append(next_x[c])
    
            sy = self.multimap_y[r] 
            # set self.most_recent_map of current cluster id to previous cluster id
            smost_recent_map[c] = self.most_recent_map[r]
        
    
        self.time_elapsed = self.time_elapsed+1

        # distance predict method
    def dist_predict(self, frame_gen): # ind: current index
        # frame gen is 
        next_frame = next(frame_gen)
        lenframe = len(next_frame)
        
        m=1000
        t = self.time_elapsed
        avx = np.mean(self.xposition[t-1])
        avy = np.mean(self.yposition[t-1])
        
        for i in range(0, lenframe):
            #if i>1:
            #    break # test out 
            ptcloud = next_frame[i].point_cloud
            xvalues=[]
            yvalues =[]
            currentdistances=[]
            for p in ptcloud:
                # extract x and y values
                xpoint =p[0]
                ypoint =p[1]
                xvalues.append(xpoint) # save to array 
                yvalues.append(ypoint)
                # calc distance
                dx1 = avx - xpoint
                dy1 = avy - ypoint
                d1 = pow(dx1, 2) + pow(dy1, 2)
                dist = pow(d1, 0.5)
                currentdistances.append(dist)
            # at the end of current cluster 
            meandistances = np.mean(currentdistances)
            if meandistances < m:
                m = meandistances
                ky = i
                hxvalues = xvalues
                hyvalues = yvalues
                
        self.xposition.append(hxvalues)
        self.yposition.append(hyvalues)
        self.trackinglist[self.time_elapsed] = ky 
        self.time_elapsed= self.time_elapsed+1
        

In [ ]:
# translate linear sum assignment to csv frame analysis:

